<a href="https://colab.research.google.com/github/mohammadgh76/Federated-Learning/blob/main/nslkdd3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import types
from numpy.lib.arraysetops import unique
import collections

import numpy as np
import tensorflow as tf
# import tensorflow_federated as tff
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

np.random.seed(0)
# tff.federated_computation(lambda: "dfkjsdjfjklfsd")()

trn = pd.read_csv("/content/kdd_train.csv")
trn.isnull().sum()
trn.nunique()
types = trn["labels"].unique()
types = types[1:]
trn["labels"].replace(to_replace=types,value="attacking",inplace = True)
c = len(trn.select_dtypes(include=["number"]).columns)

cols = ['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate']

for col in trn.select_dtypes(include=["object"]):
    print(col)

from sklearn.preprocessing import LabelEncoder
cols = ['protocol_type','service','flag','labels']
for i in cols:
    en = LabelEncoder()
    trn[i] = en.fit_transform(trn[i])


corr_matrix = trn.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
trn.drop(to_drop, axis=1, inplace=True)

# numeric_feature_names = ['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
#        'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
#        'root_shell', 'su_attempted',  'num_file_creations',
#        'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
#        'is_guest_login', 'count', 'srv_count', 'serror_rate',
#         'rerror_rate', 'same_srv_rate',
#        'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
#        'dst_host_srv_count', 'dst_host_same_srv_rate',
#        'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
#        'dst_host_srv_diff_host_rate']
# numeric_features = trn[numeric_feature_names]
# numeric_features.head()
trn.head(10)
X_trn = trn.drop(['labels'] , axis = 1).values
Y_trn = trn['labels'].values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_trn = scaler.fit_transform(X_trn)

trn = tf.convert_to_tensor(trn)
trn = tf.data.Dataset.from_tensors(trn)
train_sample = tf.convert_to_tensor(X_trn)
test_sample =  tf.convert_to_tensor(Y_trn)

NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):

    return collections.OrderedDict(
        x=tf.reshape(train_sample,[125973,-1]),
        y=tf.reshape(test_sample,[125973,-1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

preprocessed_example_dataset = preprocess(trn)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch


protocol_type
service
flag
labels


OrderedDict([('x',
              array([[-0.11024922, -0.12470616, -0.68678521, ...,  0.06997226,
                      -0.2891034 , -0.22453241],
                     [-0.11024922,  2.21931185,  0.78142753, ...,  2.36773734,
                      -0.2891034 , -0.38763462],
                     [-0.11024922, -0.12470616,  1.08730518, ..., -0.48019685,
                      -0.2891034 , -0.38763462],
                     ...,
                     [-0.11024922, -0.12470616,  1.39318283, ..., -0.48019685,
                      -0.2891034 , -0.35501418],
                     [-0.11024922, -0.12470616, -0.0750299 , ..., -0.48019685,
                      -0.2891034 , -0.38763462],
                     [-0.11024922, -0.12470616, -0.68678521, ...,  0.49068981,
                      -0.2891034 , -0.38763462]])),
             ('y',
              array([[1],
                     [1],
                     [0],
                     ...,
                     [1],
                     [0],
         